In [2]:
import pandas as pd
import numpy as np
import datetime
import copy

import sys
import time
#self function
#import Ipynb_importer
#import FeatureConstruct

In [11]:
#U用户在一个时间窗口内的浏览(1)、收藏(2)、加购物车(3)、购买(4)的行为次数，总行为次数。
#user_behavior_data: 时间窗口内的数据
def get_feature_u(user_behavior_data):
    user_behavior_data['cumcount'] = user_behavior_data.groupby(['user_id','behavior_type']).cumcount()
    #print('user_behavior_data')
    #print(user_behavior_data.head(5))
    user_behavior_data_u = user_behavior_data.drop_duplicates(['user_id','behavior_type'], 'last')[['user_id','behavior_type','cumcount']]
    #print('user_behavior_data_u-1')
    #print(user_behavior_data_u.head(5))
    user_behavior_data_u = pd.get_dummies(user_behavior_data_u['behavior_type']).join(user_behavior_data_u[['user_id','cumcount']])
    #print('user_behavior_data_u-2')
    #print(user_behavior_data_u.head(5))
    user_behavior_data_u.rename(columns = {1:'behavior_type_1',
                                                               2:'behavior_type_2',
                                                               3:'behavior_type_3',
                                                               4:'behavior_type_4'}, inplace=True)
    #print('user_behavior_data_u-3')
    #print(user_behavior_data_u.head(5))
    user_behavior_data_u['u_b1_count'] = user_behavior_data_u['behavior_type_1'] * (user_behavior_data_u['cumcount']+1)
    user_behavior_data_u['u_b2_count'] = user_behavior_data_u['behavior_type_2'] * (user_behavior_data_u['cumcount']+1)
    user_behavior_data_u['u_b3_count'] = user_behavior_data_u['behavior_type_3'] * (user_behavior_data_u['cumcount']+1)
    user_behavior_data_u['u_b4_count'] = user_behavior_data_u['behavior_type_4'] * (user_behavior_data_u['cumcount']+1)
    #print('user_behavior_data_u-4')
    #print(user_behavior_data_u.head(5))
    #这步没做什么，实质是进行了列的选取
    user_behavior_data_u = user_behavior_data_u.groupby('user_id').agg({'u_b1_count': np.sum,
                                                                                                      'u_b2_count': np.sum, 
                                                                                                      'u_b3_count': np.sum, 
                                                                                                      'u_b4_count': np.sum})
    #print('user_behavior_data_u-5')
    #print(user_behavior_data_u.head(5))

    user_behavior_data_u.reset_index(inplace = True)
    user_behavior_data_u['u_b_count'] = user_behavior_data_u[['u_b1_count',
                                                                                        'u_b2_count',
                                                                                        'u_b3_count',
                                                                                        'u_b4_count']].apply(lambda x: x.sum(), axis = 1)
    user_behavior_data_u['u_b4_rate'] = user_behavior_data_u['u_b4_count'] / user_behavior_data_u['u_b_count']
    
    # u_b4_diff_time
    user_behavior_data = user_behavior_data.sort_values(by = ['user_id', 'time'])
    user_behavior_data_u_b4_time = user_behavior_data[user_behavior_data['behavior_type'] == 4].drop_duplicates(['user_id'],'first')[['user_id','time']]
    user_behavior_data_u_b4_time.columns = ['user_id','b4_first_time']
    user_behavior_data_u_b_time = user_behavior_data.drop_duplicates(['user_id'],'first')[['user_id','time']]
    user_behavior_data_u_b_time.columns = ['user_id','b_first_time']
    user_behavior_data_u_b4_time = pd.merge(user_behavior_data_u_b_time, user_behavior_data_u_b4_time, on = ['user_id'])
    user_behavior_data_u_b4_time['u_b4_diff_time'] = user_behavior_data_u_b4_time['b4_first_time'] - user_behavior_data_u_b4_time['b_first_time']
    user_behavior_data_u_b4_time = user_behavior_data_u_b4_time[['user_id', 'u_b4_diff_time']]
    #lambda x: x.days * 24 + x.seconds//3600 这个有问题吧？
    user_behavior_data_u_b4_time['u_b4_diff_hours'] = user_behavior_data_u_b4_time['u_b4_diff_time'].apply(lambda x: x.days * 24 + x.seconds//3600) 
    user_behavior_data_u = pd.merge(user_behavior_data_u, user_behavior_data_u_b4_time, on=['user_id'], how='left')[['user_id',
                                                                                                                                                                   'u_b1_count',
                                                                                                                                                                   'u_b2_count', 
                                                                                                                                                                   'u_b3_count',
                                                                                                                                                                   'u_b4_count', 
                                                                                                                                                                   'u_b_count', 
                                                                                                                                                                   'u_b4_rate', 
                                                                                                                                                                   'u_b4_diff_hours']]
    #print('user_behavior_data_u-6')
    #print(user_behavior_data_u.head(5))
    return user_behavior_data_u

#I商品在目标日前6天的各类行为总数、总行为次数、商品的点击购买转化率、商品的点击购买平均时差、被访问人数。
def get_feature_i(user_behavior_data):
    user_behavior_data_copy = user_behavior_data
    user_behavior_data_copy['i_cumcount'] = user_behavior_data_copy.groupby(['item_id','behavior_type']).cumcount()
    user_behavior_data_i = user_behavior_data_copy.drop_duplicates(['item_id','behavior_type'], 'last')[['item_id','behavior_type','i_cumcount']]
    user_behavior_data_i = pd.get_dummies(user_behavior_data_i['behavior_type']).join(user_behavior_data_i[['item_id','i_cumcount']])
    user_behavior_data_i.rename(columns = {1:'i_behavior_type_1',
                                           2:'i_behavior_type_2',
                                           3:'i_behavior_type_3',
                                           4:'i_behavior_type_4'}, inplace=True)
    ##各类行为次数
    user_behavior_data_i['i_b1_count'] = user_behavior_data_i['i_behavior_type_1'] * (user_behavior_data_i['i_cumcount']+1)
    user_behavior_data_i['i_b2_count'] = user_behavior_data_i['i_behavior_type_2'] * (user_behavior_data_i['i_cumcount']+1)
    user_behavior_data_i['i_b3_count'] = user_behavior_data_i['i_behavior_type_3'] * (user_behavior_data_i['i_cumcount']+1)
    user_behavior_data_i['i_b4_count'] = user_behavior_data_i['i_behavior_type_4'] * (user_behavior_data_i['i_cumcount']+1)
    ##这步其实没做什么，实质是进行了列的选取
    user_behavior_data_i = user_behavior_data_i.groupby('item_id').agg({'i_b1_count': np.sum,
                                                                                                    'i_b2_count': np.sum, 
                                                                                                    'i_b3_count': np.sum, 
                                                                                                    'i_b4_count': np.sum})
    user_behavior_data_i.reset_index(inplace = True)
    ##总行为次数
    user_behavior_data_i['i_b_count'] = user_behavior_data_i[['i_b1_count',
                                                                                     'i_b2_count',
                                                                                     'i_b3_count',
                                                                                     'i_b4_count']].apply(lambda x: x.sum(), axis = 1)
    ##商品点击购买转化率
    user_behavior_data_i['i_b4_rate'] = user_behavior_data_i['i_b4_count'] / user_behavior_data_i['i_b_count']
    
    ##被访问人数
    item_user_group_size = pd.DataFrame(user_behavior_data.groupby(['item_id','user_id']).size())
    item_user_num = pd.DataFrame(item_user_group_size.groupby(['item_id']).size())
    item_user_num.rename(columns = {0:'i_user_num'},inplace=True)
    user_behavior_data_i=user_behavior_data_i.set_index('item_id')
    user_behavior_data_i = user_behavior_data_i.join(item_user_num)
    user_behavior_data_i.reset_index(inplace = True)
    ##
    ##商品点击购买平均时差（非常重要）；不用的人对同一个商品的时差不一致，需要平均。另外这个购买平均时差的特征加持在人身上效果应该会更好，
    # i_b4_diff_time
    user_behavior_data = user_behavior_data.sort_values(by=['item_id', 'time'])
    user_behavior_data_i_b4_time = user_behavior_data[user_behavior_data['behavior_type'] == 4].drop_duplicates(['item_id'], 'first')[['item_id','time']]
    user_behavior_data_i_b4_time.columns = ['item_id','b4_first_time']
    user_behavior_data_i_b_time = user_behavior_data.drop_duplicates(['item_id'], 'first')[['item_id','time']]
    user_behavior_data_i_b_time.columns = ['item_id','b_first_time']
    user_behavior_data_i_b4_time = pd.merge(user_behavior_data_i_b_time, user_behavior_data_i_b4_time, on = ['item_id'])
    user_behavior_data_i_b4_time['i_b4_diff_time']  = user_behavior_data_i_b4_time['b4_first_time'] - user_behavior_data_i_b4_time['b_first_time']
    #lambda x: x.days * 24 + x.seconds//3600 这个有问题吧？
    user_behavior_data_i_b4_time['i_b4_diff_hours'] = user_behavior_data_i_b4_time['i_b4_diff_time'].apply(lambda x: x.days * 24 + x.seconds//3600)
    user_behavior_data_i_b4_time = user_behavior_data_i_b4_time[['item_id', 'i_b4_diff_hours']]
    
    user_behavior_data_i = pd.merge(user_behavior_data_i, user_behavior_data_i_b4_time, on=['item_id'], how='left')[['item_id',
                                                                                                                                                                   'i_b1_count',
                                                                                                                                                                   'i_b2_count', 
                                                                                                                                                                   'i_b3_count',
                                                                                                                                                                   'i_b4_count', 
                                                                                                                                                                   'i_b_count', 
                                                                                                                                                                   'i_b4_rate', 
                                                                                                                                                                   'i_user_num',
                                                                                                                                                                   'i_b4_diff_hours']]  
    return user_behavior_data_i

#C类目在目标日前6天的各类行为总数、总行为次数、商品的点击购买转化率、商品的点击购买平均时差、被访问人数。
def get_feature_c(user_behavior_data):
    user_behavior_data_copy = user_behavior_data
    #df_part_1_c_b = df_part_1_u_b
    user_behavior_data_copy['c_cumcount'] = user_behavior_data_copy.groupby(['item_category','behavior_type']).cumcount()
    user_behavior_data_c = user_behavior_data_copy.drop_duplicates(['item_category','behavior_type'], 'last')[['item_category','behavior_type','c_cumcount']]
    user_behavior_data_c = pd.get_dummies(user_behavior_data_c['behavior_type']).join(user_behavior_data_c[['item_category','c_cumcount']])
    user_behavior_data_c.rename(columns = {1:'c_behavior_type_1',
                                           2:'c_behavior_type_2',
                                           3:'c_behavior_type_3',
                                           4:'c_behavior_type_4'}, inplace=True)
    ##各类行为次数
    user_behavior_data_c['c_b1_count'] = user_behavior_data_c['c_behavior_type_1'] * (user_behavior_data_c['c_cumcount']+1)
    user_behavior_data_c['c_b2_count'] = user_behavior_data_c['c_behavior_type_2'] * (user_behavior_data_c['c_cumcount']+1)
    user_behavior_data_c['c_b3_count'] = user_behavior_data_c['c_behavior_type_3'] * (user_behavior_data_c['c_cumcount']+1)
    user_behavior_data_c['c_b4_count'] = user_behavior_data_c['c_behavior_type_4'] * (user_behavior_data_c['c_cumcount']+1)
    ##这步其实没做什么，实质是进行了列的选取
    user_behavior_data_c = user_behavior_data_c.groupby('item_category').agg({'c_b1_count': np.sum,
                                                                                  'c_b2_count': np.sum, 
                                                                                  'c_b3_count': np.sum, 
                                                                                  'c_b4_count': np.sum})
    user_behavior_data_c.reset_index(inplace = True)
    ##总行为次数
    user_behavior_data_c['c_b_count'] = user_behavior_data_c[['c_b1_count',
                                                                       'c_b2_count',
                                                                       'c_b3_count',
                                                                       'c_b4_count']].apply(lambda x: x.sum(), axis = 1)
    ##商品点击购买转化率
    user_behavior_data_c['c_b4_rate'] = user_behavior_data_c['c_b4_count'] / user_behavior_data_c['c_b_count']
    
    ##被访问人数
    category_user_group_size = pd.DataFrame(user_behavior_data_copy.groupby(['item_category','user_id']).size())
    category_user_num = pd.DataFrame(category_user_group_size.groupby(['item_category']).size())
    category_user_num.rename(columns = {0:'c_user_num'},inplace=True)
    user_behavior_data_c=user_behavior_data_c.set_index(['item_category'])
    user_behavior_data_c = user_behavior_data_c.join(category_user_num)
    user_behavior_data_c.reset_index(inplace = True)

    ##商品点击购买平均时差（非常重要）；不用的人对同一个商品的时差不一致，需要平均。另外这个购买平均时差的特征加持在人身上效果应该会更好，
    user_behavior_data = user_behavior_data.sort_values(by=['item_category', 'time'])
    user_behavior_data_c_b4_time = user_behavior_data[user_behavior_data['behavior_type'] == 4].drop_duplicates(['item_category'], 'first')[['item_category','time']]
    user_behavior_data_c_b4_time.columns = ['item_category','b4_first_time']
    user_behavior_data_c_b_time = user_behavior_data.drop_duplicates(['item_category'], 'first')[['item_category','time']]
    user_behavior_data_c_b_time.columns = ['item_category','b_first_time']
    user_behavior_data_c_b_b4_time = pd.merge(user_behavior_data_c_b_time, user_behavior_data_c_b4_time, on = ['item_category'])
    user_behavior_data_c_b_b4_time['c_b4_diff_time']  = user_behavior_data_c_b_b4_time['b4_first_time'] - user_behavior_data_c_b_b4_time['b_first_time']
    user_behavior_data_c_b_b4_time['c_b4_diff_hours'] = user_behavior_data_c_b_b4_time['c_b4_diff_time'].apply(lambda x: x.days * 24 + x.seconds//3600)
    user_behavior_data_c_b_b4_time = user_behavior_data_c_b_b4_time[['item_category','c_b4_diff_hours']]
    user_behavior_data_c = pd.merge(user_behavior_data_c, user_behavior_data_c_b_b4_time, on=['item_category'], how='left')[['item_category',
                                                                                                                                                                                   'c_b1_count',
                                                                                                                                                                                   'c_b2_count', 
                                                                                                                                                                                   'c_b3_count',
                                                                                                                                                                                   'c_b4_count', 
                                                                                                                                                                                   'c_b_count', 
                                                                                                                                                                                   'c_b4_rate', 
                                                                                                                                                                                   'c_user_num',
                                                                                                                                                                                   'c_b4_diff_hours']]  
    return user_behavior_data_c

#IC商品在所属类别中的用户人数(排序)、商品在所属类别中的行为总数(排序)、商品在所属类别中的销量排序
def get_feature_ic(user_behavior_data): 
    user_behavior_data_uic = user_behavior_data.drop_duplicates(['user_id','item_id','item_category'], 'first')
    user_behavior_data_i = get_feature_i(user_behavior_data)
    user_behavior_data_i_ub_count = user_behavior_data_i[['item_id','i_user_num','i_b_count','i_b4_count']]
    
    user_behavior_data_ic_ub_count = pd.merge(user_behavior_data_uic, user_behavior_data_i_ub_count, on=['item_id'], how='left').fillna(0)
    user_behavior_data_ic_ub_count = user_behavior_data_ic_ub_count.drop_duplicates(['item_id','item_category'])

    # ic_u_rank_in_c
    user_behavior_data_ic_ub_count['ic_u_rank_in_c'] = user_behavior_data_ic_ub_count.groupby('item_category')['i_user_num'].rank(method='min',ascending=False).astype('int')
    # ic_b_rank_in_c                
    user_behavior_data_ic_ub_count['ic_b_rank_in_c'] = user_behavior_data_ic_ub_count.groupby('item_category')['i_b_count'].rank(method='min',ascending=False).astype('int')
    # ic_b4_rank_in_c
    user_behavior_data_ic_ub_count['ic_b4_rank_in_c'] = user_behavior_data_ic_ub_count.groupby('item_category')['i_b4_count'].rank(method='min',ascending=False).astype('int')

    user_behavior_data_ic = user_behavior_data_ic_ub_count[['item_id', 
                                                                                      'item_category', 
                                                                                      'ic_u_rank_in_c', 
                                                                                      'ic_b_rank_in_c', 
                                                                                      'ic_b4_rank_in_c']]

    '''user_behavior_data_copy = user_behavior_data
    ##商品在所属类别中的用户人数
    item_category_user_size = pd.DataFrame(user_behavior_data_copy.groupby(['item_id','item_category','user_id']).size())
    item_category_user_num = pd.DataFrame(item_category_user_size.groupby(['item_id','item_category']).size())
    item_category_user_num.rename(columns = {0:'ic_user_num'},inplace=True)
    #print(item_category_user_num.head(5))
    
    ##商品在所属类别中的行为总数
    item_category_behavior_size = pd.DataFrame(user_behavior_data_copy.groupby(['item_id','item_category']).size())
    item_category_behavior_size.rename(columns = {0:'ic_behavior_num'},inplace=True)
    #print('1:',item_category_behavior_size.head(5))
    
    ##商品在所属类别中的销量排序
    item_category_behavior_sales_size = pd.DataFrame(user_behavior_data_copy.groupby(['item_id','item_category','behavior_type']).size())
    item_category_behavior_sales_size = item_category_behavior_sales_size.reset_index()
    #print('2:',item_category_behavior_sales_size.head(5))
    user_behavior_data_ic = item_category_behavior_sales_size[item_category_behavior_sales_size.behavior_type==4]
    user_behavior_data_ic.rename(columns = {0:'ic_sales_num'},inplace=True)
    user_behavior_data_ic.set_index(['item_id','item_category'],inplace=True)
    #print('3:',df_part_1_ic_b_count_in_6.head(5))
    user_behavior_data_ic = user_behavior_data_ic.join(item_category_user_num,how='outer')
    user_behavior_data_ic = user_behavior_data_ic.join(item_category_behavior_size,how='outer')
    user_behavior_data_ic = user_behavior_data_ic[['ic_sales_num','ic_user_num','ic_behavior_num']].fillna(0)
    #df_part_1_ic_b_count_in_6
    user_behavior_data_ic.reset_index(inplace = True)'''
    return user_behavior_data_ic

    
#UI用户-商品对在考察日前6天的行为总数计数、用户-商品对在考察日前6天的各项行为计数、
#用户-商品对各项行为上一次发生距考察日的时差、用户商品对的行为在用户所有商品中的排序
def get_feature_ui(user_behavior_data,end_day):  
    end_day = str(end_day)
    #df_part_1_ui_b = df_part_1_u_b
    user_behavior_data_copy = user_behavior_data
    
    ui_b_count_size = pd.DataFrame(user_behavior_data_copy.groupby(['item_id','user_id','behavior_type']).size())
    ui_b_count_size = ui_b_count_size.reset_index()
    ui_b_count_size.rename(columns={0:'ui_cumcount'},inplace=True)
    ui_b_count_num = pd.get_dummies(ui_b_count_size['behavior_type']).join(ui_b_count_size[['user_id','item_id','ui_cumcount']])
    ui_b_count_num.rename(columns={1:'ui_b1_count',
                                        2:'ui_b2_count',
                                        3:'ui_b3_count',
                                        4:'ui_b4_count'},inplace=True)
    ##用户-商品对在考察日前n天的各项行为计数
    ui_b_count_num['ui_b1_count'] = ui_b_count_num['ui_b1_count'] * ui_b_count_num['ui_cumcount']
    ui_b_count_num['ui_b2_count'] = ui_b_count_num['ui_b2_count'] * ui_b_count_num['ui_cumcount']
    ui_b_count_num['ui_b3_count'] = ui_b_count_num['ui_b3_count'] * ui_b_count_num['ui_cumcount']
    ui_b_count_num['ui_b4_count'] = ui_b_count_num['ui_b4_count'] * ui_b_count_num['ui_cumcount']
    ##用户-商品对在考察日前n天的行为总数计数
    ui_b_count_num['ui_b_count']= ui_b_count_num['ui_b1_count'] + ui_b_count_num['ui_b2_count'] + ui_b_count_num['ui_b3_count'] + ui_b_count_num['ui_b4_count']
    ui_b_count_num = ui_b_count_num.groupby(['item_id','user_id']).sum()
    
    user_behavior_data_ui = ui_b_count_num[['ui_b1_count',
                                                               'ui_b2_count',
                                                               'ui_b3_count',
                                                               'ui_b4_count',
                                                               'ui_b_count']].astype(int)
    ##用户-商品对的行为在本用户所有用户-商品对中的排序
    user_behavior_data_ui['ui_b_count_rank_in_u'] = user_behavior_data_ui.groupby(['user_id'])['ui_b_count'].rank(method='min',ascending=False).astype('int')
    '''
    user_behavior_data_ui = user_behavior_data_ui.sort_values(by='ui_b_count',ascending=True)
    temp = pd.DataFrame(user_behavior_data_ui.groupby(['user_id']).cumcount())
    user_behavior_data_ui = temp.join(ui_b_count_num)
    user_behavior_data_ui.rename(columns={0:'ui_sort'},inplace=True)
    user_behavior_data_ui['ui_sort'] = user_behavior_data_ui['ui_sort']+1
    user_behavior_data_ui.reset_index(inplace = True)    
    '''
    #UI
    user_behavior_data_uic = user_behavior_data.drop_duplicates(['user_id','item_id','item_category'], 'first')
    user_behavior_data_ui = pd.merge(user_behavior_data_uic, user_behavior_data_ui, on = ['user_id','item_id'], how = 'left')
    ###UI-UC特征
    user_behavior_data_ui['ui_b_count_rank_in_uc'] = user_behavior_data_ui.groupby(['user_id','item_category'])['ui_b_count_rank_in_u'].rank(method='min',ascending=True).astype('int')
    ##用户-商品对各项行为上一次发生距考察日的时差

    # ui_b_last_time
    user_behavior_data_uic.sort_values(by=['user_id','item_id','behavior_type','time'], inplace=True)
    ub_data_ui_b_last_time = user_behavior_data_uic.drop_duplicates(['user_id','item_id','behavior_type'],'last')[['user_id','item_id','behavior_type','time']]

    ub_data_ui_b_last_time['ui_b1_last_time'] = ub_data_ui_b_last_time[ub_data_ui_b_last_time['behavior_type'] == 1]['time']
    ub_data_ui_b_last_time['ui_b2_last_time'] = ub_data_ui_b_last_time[ub_data_ui_b_last_time['behavior_type'] == 2]['time']
    ub_data_ui_b_last_time['ui_b3_last_time'] = ub_data_ui_b_last_time[ub_data_ui_b_last_time['behavior_type'] == 3]['time']
    ub_data_ui_b_last_time['ui_b4_last_time'] = ub_data_ui_b_last_time[ub_data_ui_b_last_time['behavior_type'] == 4]['time']

    ub_data_ui_b_last_time.loc[ub_data_ui_b_last_time['ui_b1_last_time'].notnull(), 'ui_b1_last_hours'] = (pd.to_datetime(end_day) - ub_data_ui_b_last_time['ui_b1_last_time'])             
    ub_data_ui_b_last_time['ui_b1_last_hours'] = ub_data_ui_b_last_time[ub_data_ui_b_last_time['ui_b1_last_hours'].notnull()]['ui_b1_last_hours'].apply(lambda x: x.days*24 + x.seconds//3600)

    ub_data_ui_b_last_time.loc[ub_data_ui_b_last_time['ui_b2_last_time'].notnull(), 'ui_b2_last_hours'] = (pd.to_datetime(end_day) - ub_data_ui_b_last_time['ui_b2_last_time'])             
    ub_data_ui_b_last_time['ui_b2_last_hours'] = ub_data_ui_b_last_time[ub_data_ui_b_last_time['ui_b2_last_hours'].notnull()]['ui_b2_last_hours'].apply(lambda x: x.days*24 + x.seconds//3600)

    ub_data_ui_b_last_time.loc[ub_data_ui_b_last_time['ui_b3_last_time'].notnull(), 'ui_b3_last_hours'] = (pd.to_datetime(end_day) - ub_data_ui_b_last_time['ui_b3_last_time'])             
    ub_data_ui_b_last_time['ui_b3_last_hours'] = ub_data_ui_b_last_time[ub_data_ui_b_last_time['ui_b3_last_hours'].notnull()]['ui_b3_last_hours'].apply(lambda x: x.days*24 + x.seconds//3600)

    ub_data_ui_b_last_time.loc[ub_data_ui_b_last_time['ui_b4_last_time'].notnull(), 'ui_b4_last_hours'] = (pd.to_datetime(end_day) - ub_data_ui_b_last_time['ui_b4_last_time'])             
    ub_data_ui_b_last_time['ui_b4_last_hours'] = ub_data_ui_b_last_time[ub_data_ui_b_last_time['ui_b4_last_hours'].notnull()]['ui_b4_last_hours'].apply(lambda x: x.days*24 + x.seconds//3600)

    ub_data_ui_b_last_time = ub_data_ui_b_last_time[['user_id',
                                                                         'item_id',
                                                                         'ui_b1_last_hours',
                                                                         'ui_b2_last_hours',
                                                                         'ui_b3_last_hours',
                                                                         'ui_b4_last_hours']] 

    ub_data_ui_b_last_time = ub_data_ui_b_last_time.groupby(['user_id', 'item_id']).agg({'ui_b1_last_hours': np.sum,
                                                                                                                         'ui_b2_last_hours': np.sum,
                                                                                                                         'ui_b3_last_hours': np.sum,
                                                                                                                         'ui_b4_last_hours': np.sum})
    ub_data_ui_b_last_time.reset_index(inplace = True)
    
    user_behavior_data_ui = pd.merge(user_behavior_data_ui, ub_data_ui_b_last_time, how='left', on=['user_id', 'item_id'])
    user_behavior_data_ui = user_behavior_data_ui[['user_id',
                                                                        'item_id',
                                                                        'ui_b1_count',
                                                                        'ui_b2_count',
                                                                        'ui_b3_count',
                                                                        'ui_b4_count',
                                                                        'ui_b_count',
                                                                        'ui_b_count_rank_in_u',
                                                                        'ui_b_count_rank_in_uc',
                                                                        'ui_b1_last_hours',
                                                                        'ui_b2_last_hours',
                                                                        'ui_b3_last_hours',
                                                                        'ui_b4_last_hours']]
    return user_behavior_data_ui

#
def get_feature_uc(user_behavior_data,end_day):  
    end_day = str(end_day)
    user_behavior_data['cumcount'] = user_behavior_data.groupby(['user_id', 'item_category', 'behavior_type']).cumcount()
    uc_b_count = user_behavior_data.drop_duplicates(['user_id','item_category','behavior_type'],'last')[['user_id','item_category','behavior_type','cumcount']]
    uc_b_count = pd.get_dummies(uc_b_count['behavior_type']).join(uc_b_count[['user_id','item_category','cumcount']])
    uc_b_count.rename(columns = {1:'behavior_type_1',
                                                2:'behavior_type_2',
                                                3:'behavior_type_3',
                                                4:'behavior_type_4'}, inplace=True)  
    uc_b_count['uc_b1_count'] = uc_b_count['behavior_type_1'] * (uc_b_count['cumcount']+1)
    uc_b_count['uc_b2_count'] = uc_b_count['behavior_type_2'] * (uc_b_count['cumcount']+1)
    uc_b_count['uc_b3_count'] = uc_b_count['behavior_type_3'] * (uc_b_count['cumcount']+1)
    uc_b_count['uc_b4_count'] = uc_b_count['behavior_type_4'] * (uc_b_count['cumcount']+1)
    
    uc_b_count = uc_b_count.groupby(['user_id', 'item_category']).agg({'uc_b1_count': np.sum,
                                                                                                 'uc_b2_count': np.sum,
                                                                                                 'uc_b3_count': np.sum,
                                                                                                 'uc_b4_count': np.sum})
    uc_b_count.reset_index(inplace = True)
    uc_b_count['uc_b_count'] = uc_b_count['uc_b1_count'] + uc_b_count['uc_b2_count'] + uc_b_count['uc_b3_count'] + uc_b_count['uc_b4_count']
    user_behavior_data_uc = uc_b_count[['user_id',
                                                        'item_category',
                                                        'uc_b1_count',
                                                        'uc_b2_count',
                                                        'uc_b3_count',
                                                        'uc_b4_count',
                                                        'uc_b_count']]
    # uc_b_count_rank_in_u
    user_behavior_data_uc['uc_b_count_rank_in_u'] = user_behavior_data_uc.groupby(['user_id'])['uc_b_count'].rank(method='min',ascending=False).astype('int')

    # uc_b_last_time
    #user_behavior_data_uic = user_behavior_data.drop_duplicates(['user_id','item_id','item_category'], 'first')
    user_behavior_data.sort_values(by=['user_id','item_category','behavior_type','time'], inplace=True)
    ub_data_uc_b_last_time = user_behavior_data.drop_duplicates(['user_id','item_category','behavior_type'],'last')[['user_id','item_category','behavior_type','time']]

    ub_data_uc_b_last_time['uc_b1_last_time'] = ub_data_uc_b_last_time[ub_data_uc_b_last_time['behavior_type'] == 1]['time']
    ub_data_uc_b_last_time['uc_b2_last_time'] = ub_data_uc_b_last_time[ub_data_uc_b_last_time['behavior_type'] == 2]['time']
    ub_data_uc_b_last_time['uc_b3_last_time'] = ub_data_uc_b_last_time[ub_data_uc_b_last_time['behavior_type'] == 3]['time']
    ub_data_uc_b_last_time['uc_b4_last_time'] = ub_data_uc_b_last_time[ub_data_uc_b_last_time['behavior_type'] == 4]['time']

    ub_data_uc_b_last_time.loc[ub_data_uc_b_last_time['uc_b1_last_time'].notnull(), 'uc_b1_last_hours'] = (pd.to_datetime(end_day) - ub_data_uc_b_last_time['uc_b1_last_time'])             
    ub_data_uc_b_last_time['uc_b1_last_hours'] = ub_data_uc_b_last_time[ub_data_uc_b_last_time['uc_b1_last_hours'].notnull()]['uc_b1_last_hours'].apply(lambda x: x.days*24 + x.seconds//3600)

    ub_data_uc_b_last_time.loc[ub_data_uc_b_last_time['uc_b2_last_time'].notnull(), 'uc_b2_last_hours'] = (pd.to_datetime(end_day) - ub_data_uc_b_last_time['uc_b2_last_time'])             
    ub_data_uc_b_last_time['uc_b2_last_hours'] = ub_data_uc_b_last_time[ub_data_uc_b_last_time['uc_b2_last_hours'].notnull()]['uc_b2_last_hours'].apply(lambda x: x.days*24 + x.seconds//3600)

    ub_data_uc_b_last_time.loc[ub_data_uc_b_last_time['uc_b3_last_time'].notnull(), 'uc_b3_last_hours'] = (pd.to_datetime(end_day) - ub_data_uc_b_last_time['uc_b3_last_time'])             
    ub_data_uc_b_last_time['uc_b3_last_hours'] = ub_data_uc_b_last_time[ub_data_uc_b_last_time['uc_b3_last_hours'].notnull()]['uc_b3_last_hours'].apply(lambda x: x.days*24 + x.seconds//3600)

    ub_data_uc_b_last_time.loc[ub_data_uc_b_last_time['uc_b4_last_time'].notnull(), 'uc_b4_last_hours'] = (pd.to_datetime(end_day) - ub_data_uc_b_last_time['uc_b4_last_time'])             
    ub_data_uc_b_last_time['uc_b4_last_hours'] = ub_data_uc_b_last_time[ub_data_uc_b_last_time['uc_b4_last_hours'].notnull()]['uc_b4_last_hours'].apply(lambda x: x.days*24 + x.seconds//3600)

    ub_data_uc_b_last_time = ub_data_uc_b_last_time[['user_id',
                                                                             'item_category',
                                                                             'uc_b1_last_hours',
                                                                             'uc_b2_last_hours',
                                                                             'uc_b3_last_hours',
                                                                             'uc_b4_last_hours']] 

    ub_data_uc_b_last_time = ub_data_uc_b_last_time.groupby(['user_id', 'item_category']).agg({'uc_b1_last_hours': np.sum,
                                                                                                                                   'uc_b2_last_hours': np.sum,
                                                                                                                                   'uc_b3_last_hours': np.sum,
                                                                                                                                   'uc_b4_last_hours': np.sum})
    ub_data_uc_b_last_time.reset_index(inplace = True)

    # merge for generation of f_UC_part_1
    user_behavior_data_uc = pd.merge(user_behavior_data_uc, ub_data_uc_b_last_time, how='left', on=['user_id', 'item_category'])
    user_behavior_data_uc = user_behavior_data_uc[['user_id',
                                                                         'item_category',
                                                                         'uc_b1_count',
                                                                         'uc_b2_count',
                                                                         'uc_b3_count',
                                                                         'uc_b4_count',
                                                                         'uc_b_count',
                                                                         'uc_b_count_rank_in_u',
                                                                         'uc_b1_last_hours',
                                                                         'uc_b2_last_hours',
                                                                         'uc_b3_last_hours',
                                                                         'uc_b4_last_hours']]
    return user_behavior_data_uc

In [10]:
def feature_construct(start_day, end_day, feature_window_size):
    start_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    #build file name
    second_end_day = (end_day - datetime.timedelta(days=1))    
    start_day_str = str(start_day.date())
    end_day_str = str(end_day.date())
    second_end_day_str = str(second_end_day.date())
    print('feature construct from %s  to %s start time:'%(start_day_str,end_day_str), start_time)
    name = start_day_str + "to" + second_end_day_str
    
    #input file name
    part_data_file_name = "../DataSet/DevideData/"+"part_data"+name+".csv"
    #part_data_label_file_name =  "../DataSet/"+"part_data_label"+end_day_str+".csv"
    part_data_label_filename =  "../DataSet/DevideData/"+"part_data_uicl"+end_day_str+".csv"
    user_behavior_data = pd.read_csv(part_data_file_name,names=['time','user_id','item_id','behavior_type','item_category'],header=None,parse_dates=[0])
    user_behavior_data.columns = ['time','user_id','item_id','behavior_type','item_category']
    
    
    #output file name
    train_data_filename = "../DataSet/Feature/"+"part_data_uiclf"+end_day_str+".csv"
    #print(user_behavior_data.head(5))
    #user_behavior_data_uic_label = pd.read_csv(part_data_label_filename,names=['time','user_id','item_id','behavior_type','item_category'],header=None)
    user_behavior_data_uic_label = pd.read_csv(part_data_label_filename)
    #print(user_behavior_data_uic_label)
    u=list()
    i_i=list()
    c=list()
    ic=list()
    ui=list()
    uc=list()
    for i in list([4,2,1]): 
        j=0  
        start_day = (end_day - datetime.timedelta(days=i))   
        end_day = end_day
        user_behavior_data = user_behavior_data[user_behavior_data['time'] >= start_day]
        
        #提取特征部分
        u.append(get_feature_u(copy.deepcopy(user_behavior_data)))
        i_i.append(get_feature_i(copy.deepcopy(user_behavior_data)))
        c.append(get_feature_c(copy.deepcopy(user_behavior_data)))
        ic.append(get_feature_ic(copy.deepcopy(user_behavior_data)))
        ui.append(get_feature_ui(copy.deepcopy(user_behavior_data),end_day))
        uc.append(get_feature_uc(copy.deepcopy(user_behavior_data),end_day))
        j+=1
        
    feature_construct_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print('feature construct  time:', feature_construct_time)
    for i in list(range(len(u)))[1:]:
        u[0] = pd.merge(u[0], u[i], on = ['user_id'], how = 'left').fillna(0)
        i_i[0] = pd.merge(i_i[0], i_i[i], on = ['item_id'], how = 'left').fillna(0)
        c[0] = pd.merge(c[0], c[i], on = ['item_category'], how = 'left').fillna(0)
        ic[0] = pd.merge(ic[0], ic[i], on = ['item_id','item_category'], how = 'left').fillna(0)
        ui[0] = pd.merge(ui[0], ui[i], on = ['user_id','item_id'], how = 'left').fillna(0)
        uc[0]= pd.merge(uc[0], uc[i], on = ['user_id','item_category'], how = 'left').fillna(0)
        
    feature_construct_join_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print('feature construct join time:', feature_construct_join_time)
    #这个位置填充的合适的值是什么？   
    feature_data = pd.merge(user_behavior_data_uic_label, u[0], how='left', on=['user_id'])
    feature_data = pd.merge(feature_data, i_i[0],  how='left', on=['item_id'])
    feature_data = pd.merge(feature_data, c[0],  how='left', on=['item_category'])
    feature_data = pd.merge(feature_data, ic[0], how='left', on=['item_id','item_category'])
    feature_data = pd.merge(feature_data, ui[0], how='left', on=['user_id','item_id'])  
    feature_data = pd.merge(feature_data, uc[0], how='left', on=['user_id','item_category'])

    feature_data.fillna(-1, inplace=True)# 一个重要的位置###############
    
    train_data = feature_data
    end_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    train_data.to_csv(train_data_filename,index=False)
    print('end time:', end_time)